In [1]:
# coding: utf-8
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Input, Lambda
from keras.layers import merge 
from keras.layers.normalization import BatchNormalization
from keras import backend as K
from keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping, ReduceLROnPlateau
from keras.models import load_model
from keras.models import model_from_json
from keras.optimizers import RMSprop
import keras.backend.tensorflow_backend as KTF
from keras.utils.np_utils import to_categorical
import tensorflow as tf

Using TensorFlow backend.


In [2]:
from pelops.datasets.featuredataset import FeatureDataset
from pelops.datasets.veri import VeriDataset
from pelops.experiment_api.experiment import ExperimentGenerator
from pelops.analysis import analysis
from pelops.analysis.camerautil import get_match_id, make_good_bad
import pelops.utils as utils
import numpy as np

import os
import glob
import random
import multiprocessing as mp
import queue
import threading
import numpy as np
import datetime

In [3]:
# get a GPU session and reserve memory
def get_session(gpu_fraction=0.3):
    '''Assume that you have 6GB of GPU memory and want to allocate ~2GB'''

    num_threads = os.environ.get('OMP_NUM_THREADS')
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=gpu_fraction)

    if num_threads:
        return tf.Session(config=tf.ConfigProto(
            gpu_options=gpu_options, intra_op_parallelism_threads=num_threads))
    else:
        return tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))


# load an image from disk
def load_image(img_path):
    img = image.load_img(img_path, target_size=(299, 299))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x


# use an image with a model to get features
def image_features(img, model, length=2048):
    features = np.zeros((1, length), dtype=np.float16)
    #model = Model(input=base_model.input, output=base_model.get_layer('flatten_1').output)
    predictions = model.predict(img)
    return predictions

In [4]:
def save_model_workaround(model, model_output_file, weights_output_file):
    print('saving model   to {}'.format(model_output_file))
    print('saving weignts to {}'.format(weights_output_file))
    # serialize model to JSON
    model_json = model.to_json()
    with open(model_output_file, 'w') as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights(weights_output_file)


def load_model_workaround(model_output_file, weights_output_file):
    # load json and create model
    json_file = open(model_output_file, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights(weights_output_file)
    return loaded_model

In [5]:
def s_distance(vects):
    x, y = vects
    s = K.abs(x - y)
    #s =  K.sqrt(K.square(x - y))
    return (s)
    #return K.squeeze(x,1) - K.squeeze(y,1)
    
def s_shape(shapes):
    shape = list(shapes)
    outshape = (shape[0])
    return tuple(outshape)


In [6]:
def make_examples(gen,examples):
    left = list()
    right = list()
    truth = list()
    
    for _ in range(examples):
        cameras = gen.generate()
        match_id = get_match_id(cameras)
        goods, bads = make_good_bad(cameras,match_id)
        left.append(goods[0])
        right.append(goods[1])
        left.append(goods[1])
        right.append(goods[0])
        
        left.append(bads[0])
        right.append(bads[1])
        left.append(bads[1])
        left.append(bads[0])
        
        truth.append(1)
        truth.append(1)
        
        truth.append(0)
        truth.append(0)

    return(left,right,truth)

def examples_2_images(left,right):
    length = len(left)
    left_images = np.zeros((length,299,299,3))
    right_images = np.zeros((length,299,299,3))
    for idx,work in enumerate(left):
        filename = work.filepath
        left_images[idx] = load_image(filename)
    for idx,work in enumerate(right):
        filename = work.filepath
        right_images[idx] = load_image(filename)
    return left_images, right_images

In [7]:
#do the keras setup

In [8]:
KTF.set_session(get_session(.90))

In [9]:
base_model = ResNet50(weights='imagenet', include_top=False)

In [10]:
input_left = Input(shape=(299, 299, 3))
input_right = Input(shape=(299, 299, 3))

In [11]:
processed_left = base_model(input_left)
processed_right = base_model(input_right)

In [12]:
#use a distance measure for making the join
#siamese_join = Lambda(s_distance,
#                  output_shape=s_shape)([processed_left, processed_right])

In [13]:
#join by slapping vectors together
siamese_join = merge([processed_left, processed_right], mode='concat')

In [14]:
siamese_join.get_shape()

TensorShape([Dimension(None), Dimension(1), Dimension(1), Dimension(4096)])

In [15]:
my_layer = GlobalAveragePooling2D()(siamese_join)

In [16]:
my_layer = Dense(4096, activation='relu')(my_layer)

In [17]:
my_layer = BatchNormalization()(my_layer)

In [18]:
my_layer = Dense(2048, activation='relu')(my_layer)

In [19]:
my_layer = BatchNormalization()(my_layer)

In [20]:
my_layer = Dense(2048, activation='relu')(my_layer)

In [21]:
num_training_classes=2

In [22]:
predictions = Dense(num_training_classes, activation='softmax')(my_layer)

In [23]:
model = Model([input_left, input_right], output=predictions)

In [24]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [25]:
#get ready to make data and try the model

In [26]:
#set some constants
ITEMSPERCAMERA = 2
YRANDOM=1024
CAMERAS=2
DROPPED=0
CMC=100
EXPERIMENTS=2000

In [27]:
veri_validate = VeriDataset('/local_data/dgrossman/VeRi',set_type=utils.SetType.TEST.value)
veri_train = VeriDataset('/local_data/dgrossman/VeRi',set_type = utils.SetType.TRAIN.value)

In [28]:
expGen_validate = ExperimentGenerator(veri_validate, CAMERAS, ITEMSPERCAMERA, DROPPED, YRANDOM)
expGen_train = ExperimentGenerator(veri_train, CAMERAS, ITEMSPERCAMERA, DROPPED, YRANDOM)

In [29]:
left_train,right_train,truth_train = make_examples(expGen_train,EXPERIMENTS)

In [30]:
left_validate,right_validate,truth_validate = make_examples(expGen_validate,EXPERIMENTS)

In [ ]:
left_images_train, right_images_train = examples_2_images(left_train,right_train)
left_images_validate, right_images_validate = examples_2_images(left_validate,right_validate)

In [ ]:
epochs = 10
frozen_history = model.fit([left_images_train, right_images_train], 
                         np.array(to_categorical(truth_train)),
                         validation_data=([left_images_validate,
                                           right_images_validate], 
                                          np.array(to_categorical(truth_validate))),
                         batch_size=16,
                         nb_epoch=epochs)

Train on 4000 samples, validate on 4000 samples
Epoch 1/10
4000/4000 [==============================] - 292s - loss: 2.1791 - acc: 0.5192 - val_loss: 5.1585 - val_acc: 0.4900

In [ ]:
full_history.history

In [ ]:
for layer in model.layers:
    try:
        if not layer.trainable:
            layer.trainable = True
    except:
        print('error layer is stuck:', layer.name)

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
epochs = 1
full_history = model.fit([left_images_train, right_images_train], 
                         np.array(to_categorical(truth_train)),
                         validation_data=([left_images_validate,
                                           right_images_validate], 
                                          np.array(to_categorical(truth_validate))),
                         batch_size=16,
                         nb_epoch=epochs)

Train on 4000 samples, validate on 4000 samples
Epoch 1/1
 240/4000 [>.............................] - ETA: 171s - loss: 0.1130 - acc: 0.9625

In [43]:
save_model_workaround(model,
                      '/local_data/dgrossman/model_save_dir/VeRi-siamese-concat.model.json',
                      '/local_data/dgrossman/model_save_dir/VeRi-siamese-concat.weights.hdf5')


saving model   to /local_data/dgrossman/model_save_dir/VeRi-siamese-concat.model.json
saving weignts to /local_data/dgrossman/model_save_dir/VeRi-siamese-concat.weights.hdf5
